# Evaluation Framework for Capstone Projects

**Module:** 16 - Capstone Project (Phase 4)
**Time:** 4-6 hours
**Difficulty:** ⭐⭐⭐⭐

---

## 🎯 Learning Objectives

By the end of this notebook, you will:
- [ ] Understand evaluation best practices for AI systems
- [ ] Create custom evaluation metrics
- [ ] Build automated benchmark suites
- [ ] Use LLM-as-judge for quality assessment
- [ ] Generate evaluation reports

---

## 🧒 ELI5: Why Evaluation Matters

> **Imagine you baked a cake** but never tasted it before serving:
>
> - Is it sweet enough?
> - Is it cooked through?
> - Would guests like it?
>
> **Evaluation is tasting your AI system.** Without it, you don't know if:
> - The model gives correct answers
> - The system is fast enough
> - Users will find it helpful
>
> Good evaluation tells you: "This works well" or "Fix this part."

---

## Part 1: Evaluation Fundamentals

In [ ]:
# Evaluation Framework Foundation

import torch
from datetime import datetime
from typing import List, Dict, Any, Callable, Optional
from dataclasses import dataclass, field
import json
import time
import statistics

print("🎯 CAPSTONE EVALUATION FRAMEWORK")
print("="*60)
print(f"Date: {datetime.now().strftime('%Y-%m-%d %H:%M')}")

@dataclass
class EvalSample:
    """A single evaluation sample."""
    id: str
    input: str
    expected: str = ""
    category: str = "general"
    difficulty: str = "medium"
    metadata: Dict[str, Any] = field(default_factory=dict)

@dataclass
class EvalResult:
    """Result of evaluating a sample."""
    sample_id: str
    input: str
    expected: str
    actual: str
    scores: Dict[str, float]
    latency_ms: float
    metadata: Dict[str, Any] = field(default_factory=dict)

@dataclass
class EvalReport:
    """Complete evaluation report."""
    name: str
    timestamp: datetime
    num_samples: int
    aggregate_scores: Dict[str, float]
    by_category: Dict[str, Dict[str, float]]
    by_difficulty: Dict[str, Dict[str, float]]
    latency_stats: Dict[str, float]
    results: List[EvalResult]
    
    def to_markdown(self) -> str:
        """Generate markdown report."""
        lines = [
            f"# Evaluation Report: {self.name}",
            f"Generated: {self.timestamp.strftime('%Y-%m-%d %H:%M:%S')}",
            f"Samples: {self.num_samples}",
            "",
            "## Aggregate Scores",
            "| Metric | Score |",
            "|--------|-------|",
        ]
        
        for metric, score in self.aggregate_scores.items():
            lines.append(f"| {metric} | {score:.4f} |")
        
        lines.extend([
            "",
            "## Latency Stats",
            f"- Mean: {self.latency_stats.get('mean', 0):.1f} ms",
            f"- P50: {self.latency_stats.get('p50', 0):.1f} ms",
            f"- P95: {self.latency_stats.get('p95', 0):.1f} ms",
            f"- Max: {self.latency_stats.get('max', 0):.1f} ms",
        ])
        
        if self.by_category:
            lines.extend(["", "## By Category"])
            for cat, scores in self.by_category.items():
                lines.append(f"\n### {cat}")
                for metric, score in scores.items():
                    lines.append(f"- {metric}: {score:.4f}")
        
        return "\n".join(lines)

print("✅ Evaluation data structures defined")

---

## Part 2: Metric Functions

In [ ]:
# Common Evaluation Metrics

def exact_match(expected: str, actual: str) -> float:
    """Exact string match (case-insensitive)."""
    return 1.0 if expected.lower().strip() == actual.lower().strip() else 0.0

def contains_answer(expected: str, actual: str) -> float:
    """Check if expected answer is contained in actual."""
    return 1.0 if expected.lower() in actual.lower() else 0.0

def keyword_coverage(expected: str, actual: str) -> float:
    """Check coverage of keywords from expected in actual."""
    # Extract keywords (words longer than 3 chars)
    expected_words = set(w.lower() for w in expected.split() if len(w) > 3)
    if not expected_words:
        return 1.0
    
    actual_lower = actual.lower()
    matches = sum(1 for w in expected_words if w in actual_lower)
    return matches / len(expected_words)

def response_length_score(expected: str, actual: str, tolerance: float = 0.5) -> float:
    """Score based on response length similarity."""
    if not expected:
        return 1.0 if len(actual) > 0 else 0.0
    
    ratio = len(actual) / len(expected)
    
    # Score drops as ratio deviates from 1.0
    if ratio < (1 - tolerance):
        return ratio / (1 - tolerance)
    elif ratio > (1 + tolerance * 2):
        return max(0, 1 - (ratio - 1 - tolerance * 2) / 2)
    else:
        return 1.0

def semantic_similarity(expected: str, actual: str, model=None) -> float:
    """
    Compute semantic similarity using sentence embeddings.
    
    Requires sentence-transformers.
    """
    if model is None:
        try:
            from sentence_transformers import SentenceTransformer
            model = SentenceTransformer('all-MiniLM-L6-v2', device='cuda')
        except ImportError:
            print("⚠️ sentence-transformers not installed")
            return keyword_coverage(expected, actual)  # Fallback
    
    embeddings = model.encode([expected, actual])
    similarity = float(embeddings[0] @ embeddings[1] / 
                      (sum(embeddings[0]**2)**0.5 * sum(embeddings[1]**2)**0.5))
    return max(0, similarity)

# Metric registry
METRICS = {
    "exact_match": exact_match,
    "contains_answer": contains_answer,
    "keyword_coverage": keyword_coverage,
    "length_score": response_length_score,
}

print("✅ Metric functions defined")
print(f"\nAvailable metrics: {list(METRICS.keys())}")

---

## Part 3: LLM-as-Judge

In [ ]:
# LLM-as-Judge Evaluation

class LLMJudge:
    """
    Use an LLM to judge response quality.
    """
    
    def __init__(self, model_name: str = "meta-llama/Llama-3.3-8B-Instruct"):
        self.model_name = model_name
        self._model = None
        self._tokenizer = None
    
    def _load(self):
        if self._model is not None:
            return
        
        from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
        
        print(f"Loading judge model: {self.model_name}")
        
        bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_compute_dtype=torch.bfloat16,
        )
        
        self._model = AutoModelForCausalLM.from_pretrained(
            self.model_name,
            quantization_config=bnb_config,
            device_map="auto",
        )
        self._tokenizer = AutoTokenizer.from_pretrained(self.model_name)
    
    def judge(
        self,
        question: str,
        expected: str,
        actual: str,
        criteria: List[str] = None
    ) -> Dict[str, Any]:
        """
        Judge a response using an LLM.
        
        Args:
            question: The original question
            expected: Expected/reference answer
            actual: Model's actual response
            criteria: List of evaluation criteria
            
        Returns:
            Dict with scores and reasoning
        """
        self._load()
        
        criteria = criteria or ["accuracy", "completeness", "clarity"]
        criteria_str = "\n".join([f"- {c}" for c in criteria])
        
        prompt = f"""You are an expert evaluator. Rate the following response.

Question: {question}

Reference Answer: {expected}

Response to Evaluate: {actual}

Evaluate on these criteria:
{criteria_str}

For each criterion, give a score from 1-5 and brief explanation.
Then give an overall score from 1-5.

Format your response as:
CRITERION: [name]
SCORE: [1-5]
REASON: [explanation]

OVERALL: [1-5]
SUMMARY: [brief overall assessment]"""
        
        messages = [
            {"role": "system", "content": "You are a fair and thorough evaluator."},
            {"role": "user", "content": prompt}
        ]
        
        text = self._tokenizer.apply_chat_template(messages, tokenize=False)
        inputs = self._tokenizer(text, return_tensors="pt").to("cuda")
        
        with torch.no_grad():
            outputs = self._model.generate(
                **inputs,
                max_new_tokens=512,
                temperature=0.3,
                do_sample=True,
            )
        
        response = self._tokenizer.decode(
            outputs[0][inputs["input_ids"].shape[1]:],
            skip_special_tokens=True
        )
        
        # Parse response
        return self._parse_judgement(response, criteria)
    
    def _parse_judgement(self, response: str, criteria: List[str]) -> Dict[str, Any]:
        """Parse the judge's response."""
        import re
        
        result = {
            "raw_response": response,
            "scores": {},
            "reasons": {},
            "overall": 0.0,
            "summary": "",
        }
        
        # Parse criterion scores
        for criterion in criteria:
            pattern = rf"{criterion}.*?SCORE:\s*(\d)"  
            match = re.search(pattern, response, re.IGNORECASE | re.DOTALL)
            if match:
                result["scores"][criterion] = int(match.group(1)) / 5.0
        
        # Parse overall
        overall_match = re.search(r"OVERALL:\s*(\d)", response, re.IGNORECASE)
        if overall_match:
            result["overall"] = int(overall_match.group(1)) / 5.0
        
        # Parse summary
        summary_match = re.search(r"SUMMARY:\s*(.+?)(?:\n|$)", response, re.IGNORECASE)
        if summary_match:
            result["summary"] = summary_match.group(1).strip()
        
        return result

print("✅ LLM Judge defined")

---

## Part 4: Evaluation Runner

In [ ]:
# Complete Evaluation Runner

class EvaluationRunner:
    """
    Runs evaluations on AI systems.
    """
    
    def __init__(
        self,
        metrics: List[str] = None,
        use_llm_judge: bool = False,
    ):
        self.metrics = metrics or ["keyword_coverage", "length_score"]
        self.use_llm_judge = use_llm_judge
        self.judge = LLMJudge() if use_llm_judge else None
    
    def evaluate(
        self,
        system_fn: Callable[[str], str],
        samples: List[EvalSample],
        name: str = "evaluation"
    ) -> EvalReport:
        """
        Run evaluation on a set of samples.
        
        Args:
            system_fn: Function that takes input and returns output
            samples: Evaluation samples
            name: Name for this evaluation
            
        Returns:
            Complete evaluation report
        """
        print(f"\n🔄 Running evaluation: {name}")
        print(f"   Samples: {len(samples)}")
        print(f"   Metrics: {self.metrics}")
        
        results = []
        latencies = []
        
        for i, sample in enumerate(samples):
            # Generate response
            start = time.time()
            try:
                actual = system_fn(sample.input)
            except Exception as e:
                actual = f"ERROR: {e}"
            latency_ms = (time.time() - start) * 1000
            latencies.append(latency_ms)
            
            # Compute metrics
            scores = {}
            for metric_name in self.metrics:
                if metric_name in METRICS:
                    scores[metric_name] = METRICS[metric_name](sample.expected, actual)
            
            # LLM judge
            if self.use_llm_judge and self.judge:
                judgement = self.judge.judge(sample.input, sample.expected, actual)
                scores["llm_judge"] = judgement["overall"]
            
            results.append(EvalResult(
                sample_id=sample.id,
                input=sample.input,
                expected=sample.expected,
                actual=actual,
                scores=scores,
                latency_ms=latency_ms,
                metadata={"category": sample.category, "difficulty": sample.difficulty}
            ))
            
            # Progress
            if (i + 1) % 10 == 0:
                print(f"   Processed {i+1}/{len(samples)}")
        
        # Aggregate results
        report = self._create_report(name, results, latencies)
        
        print(f"\n✅ Evaluation complete")
        print(f"   Overall scores: {report.aggregate_scores}")
        
        return report
    
    def _create_report(
        self,
        name: str,
        results: List[EvalResult],
        latencies: List[float]
    ) -> EvalReport:
        """Create report from results."""
        # Aggregate scores
        all_metrics = set()
        for r in results:
            all_metrics.update(r.scores.keys())
        
        aggregate = {}
        for metric in all_metrics:
            scores = [r.scores.get(metric, 0) for r in results]
            aggregate[metric] = statistics.mean(scores) if scores else 0
        
        # By category
        by_category = {}
        categories = set(r.metadata.get("category", "general") for r in results)
        for cat in categories:
            cat_results = [r for r in results if r.metadata.get("category") == cat]
            by_category[cat] = {}
            for metric in all_metrics:
                scores = [r.scores.get(metric, 0) for r in cat_results]
                by_category[cat][metric] = statistics.mean(scores) if scores else 0
        
        # By difficulty
        by_difficulty = {}
        difficulties = set(r.metadata.get("difficulty", "medium") for r in results)
        for diff in difficulties:
            diff_results = [r for r in results if r.metadata.get("difficulty") == diff]
            by_difficulty[diff] = {}
            for metric in all_metrics:
                scores = [r.scores.get(metric, 0) for r in diff_results]
                by_difficulty[diff][metric] = statistics.mean(scores) if scores else 0
        
        # Latency stats
        sorted_latencies = sorted(latencies)
        latency_stats = {
            "mean": statistics.mean(latencies),
            "p50": sorted_latencies[len(sorted_latencies) // 2],
            "p95": sorted_latencies[int(len(sorted_latencies) * 0.95)],
            "max": max(latencies),
            "min": min(latencies),
        }
        
        return EvalReport(
            name=name,
            timestamp=datetime.now(),
            num_samples=len(results),
            aggregate_scores=aggregate,
            by_category=by_category,
            by_difficulty=by_difficulty,
            latency_stats=latency_stats,
            results=results
        )

print("✅ Evaluation runner ready")

---

## Part 5: Example Usage

In [ ]:
# Example: Running an evaluation

# Create sample evaluation dataset
example_samples = [
    EvalSample(
        id="1",
        input="What is the capital of France?",
        expected="Paris is the capital of France.",
        category="factual",
        difficulty="easy"
    ),
    EvalSample(
        id="2",
        input="Explain how photosynthesis works.",
        expected="Photosynthesis is the process by which plants convert sunlight, water, and carbon dioxide into glucose and oxygen.",
        category="explanation",
        difficulty="medium"
    ),
    EvalSample(
        id="3",
        input="What are the pros and cons of renewable energy?",
        expected="Pros: sustainable, clean, reduces emissions. Cons: intermittent, high initial cost, land use.",
        category="analysis",
        difficulty="hard"
    ),
]

# Mock system function (replace with your actual system)
def mock_system(input_text: str) -> str:
    """Mock system for demonstration."""
    if "capital" in input_text.lower():
        return "Paris is the capital city of France, located in the north of the country."
    elif "photosynthesis" in input_text.lower():
        return "Photosynthesis converts sunlight into energy for plants using water and CO2."
    elif "renewable" in input_text.lower():
        return "Renewable energy has benefits like being sustainable but drawbacks like intermittency."
    else:
        return "I don't have information about that."

# Run evaluation
runner = EvaluationRunner(
    metrics=["keyword_coverage", "contains_answer", "length_score"],
    use_llm_judge=False  # Set True if you want LLM evaluation
)

report = runner.evaluate(
    system_fn=mock_system,
    samples=example_samples,
    name="Demo Evaluation"
)

# Print report
print("\n" + "="*60)
print(report.to_markdown())

---

## 🎉 Checkpoint

You now have a complete evaluation framework:

- ✅ Evaluation data structures
- ✅ Multiple metric functions
- ✅ LLM-as-judge capability
- ✅ Automated evaluation runner
- ✅ Report generation

### Applying to Your Project

1. Create evaluation samples specific to your domain
2. Choose or create metrics that matter for your use case
3. Run evaluations during development
4. Compare against baselines
5. Include results in your technical report

---

In [ ]:
# Cleanup
import gc
if torch.cuda.is_available():
    torch.cuda.empty_cache()
gc.collect()

print("✅ Ready to evaluate your capstone project!")